# Итоговый проект по программе "Специалист по Data Science"

## Тема: Определение вероятности покупки товара на основе данных о клиенте и его покупательской истории с помощью методов машинного обучения

**Цель проекта:** Продемонстрировать освоение ключевых методов анализа данных и их приминение на практике на основе задачи определения вероятности совершения покупки на основе данных о клиенте и его покупательской истории


**Задачи проекта:** 
1. Провести предобработку и исследовательский анализ данных полученного датасета
2. Составить портрет покупателя
3. Провести кластеризацию покупателей
4. Выбрать и обучить модель определения вероятности покупки товара


**Исходные данные:**
* Датасет ["Superstore Marketing Campaign Dataset"](https://www.kaggle.com/datasets/ahsan81/superstore-marketing-campaign-dataset)

**Содержание отчета:**
* 1. Загрузка датасета и знакомство с данными
* 2. Предобработка данных
     * 2.1 Типы данных
     * 2.2 Пропуски данных
     * 2.3 Дубликаты данных
     * 2.4 Дообогащение данных
     * 2.5. Выводы по предобработке данных
* 3. Исследовательский анализ данных
* 4. Портрет покупателя
* 5. Кластеризация покупателей
* 6. Выбор и обучение модели для определения вероятности покупки товара
* 7. Общие выводы по результатам работы

## 1. Загрузка датасета и знакомство с данными

In [6]:
# Импорт необходимых библиотек

import pandas as pd

import warnings

# Настройка отоброжения датасета в тетрадке
pd.set_option('display.max_columns', None) # вывод результатов без сокращения количества столбцов

# Отключаем предупреждения
warnings.filterwarnings('ignore')

In [3]:
# Загрузка датасета

data = pd.read_csv('superstore_data.csv')

In [7]:
# Откроем датасет и посмотрим на данные

data.head()

,Id,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Response,Complain
0,1826,1970,Graduation,Divorced,84835.0,0,0,6/16/2014,0,189,104,379,111,189,218,1,4,4,6,1,1,0
1,1,1961,Graduation,Single,57091.0,0,0,6/15/2014,0,464,5,64,7,0,37,1,7,3,7,5,1,0
2,10476,1958,Graduation,Married,67267.0,0,1,5/13/2014,0,134,11,59,15,2,30,1,3,2,5,2,0,0
3,1386,1967,Graduation,Together,32474.0,1,1,11/5/2014,0,10,0,1,0,0,0,1,1,0,2,7,0,0
4,5371,1989,Graduation,Single,21474.0,1,0,8/4/2014,0,6,16,24,11,0,34,2,3,1,2,7,1,0


In [8]:
# Посмотрим общую информацию о датасете

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

В датасете содиржится меформация о 2240 клиентов с 22 признаками. К датасету прилагалось следующее описание:

* ``Id`` - уникальный идентификатор клиента
* ``Year_Birth`` - год рождения клиента
* ``Education`` - уровень образования клиента
* ``Marital_Status`` - семейное положение клиента
* ``Income`` - годовой доход домохозяйства клиента
* ``Kidhome`` - количество маленьких детей у клиента
* ``Teenhome`` - количество детей-подростков 
* ``Dt_Customer`` - дата регистрации клиента
* ``Recency`` - количество дней с последней покупки
* ``MntWines`` - сумма, потраченная на вино за последние два года
* ``MntFruits`` -  сумма, потраченная на фрукты за последние два года
* ``MntMeatProducts`` -  сумма, потраченная на мясо за последние два года
* ``MntFishProducts`` -  сумма, потраченная на рыбу за последние два года
* ``MntSweetProducts`` -  сумма, потраченная на сладости за последние два года
* ``MntGoldProds`` -  сумма, потраченная на "золотые продукты" за последние два года. "Золотые продукты" - это продукты, размещаемые на "золотой полке" — это полка, расположенная на уровне глаз покупателей, на высоте от 1,2 до 1,7 метра от пола (с учётом среднего роста взрослого человека). В детских магазинах золотая полка находится на уровне от 0,3 до 1 метра от пола. 
* ``NumDealsPurchases`` - количество покупок со скидкой
* ``NumWebPurchases`` - количество покупок совершенных на сайте компании
* ``NumCatalogPurchases`` - количество покупок, совершенных с помощью каталога (покупка товаров, которые должны быть отправлены по почте)
* ``NumStorePurchases`` - количество покупок совершенных непосредственно в магазине
* ``NumWebVisitsMonth`` - количество посещений сайта компании за последний месяц
* ``Response`` - ответ клиента (целевая переменная), 1 - клиент принял предложение о покупке, 0 - не принял
* ``Complain`` - жалобы клиента, 1 - жалобы были, 0 - жалоб не было

## 2. Предобработка данных

Исходя из проведенного обзора данных необходимо выполнить:
* Привести типы данных в соответствие
* Проверить пропуски в датасете
* Проверить наличие явных и неявных дубликатов
* По возможности дообогатить данные
* Удалить столбцы не несущие смысла для дальнейшего анализа

In [35]:
# Для дальнейшего удобства приведем названия столбцов к нижнему регистру

# Получим список наименований столбцов
columns_list = data.columns.to_list()

# Переведем все названия в нижний регистр
lower_col_names = list(map(lambda x: x.lower(), columns_list))

# Заменим название столбцов на новое (в нижнем регистре)
data.columns = lower_col_names

### 2.1. Типы данных

Поле ``dt_сustomer`` содержит дату регистрации клиента и  представлена в строковом виде - необходимо привести к типу даты.

In [38]:
# Преобразуем Dt_Customer к типу даты

data['dt_customer'] = pd.to_datetime(data['dt_customer'])

В остальных полях типы данных указаны корректно.

### 2.2. Пропуски данных

In [39]:
# Проверим наличие NAN и NULL в датасете

print(20*'==')
print('NULLS')
print(data.isnull().sum())
print(20*'==')
print('NANs')
print(data.isna().sum())
print(20*'==')

NULLS
id                      0
year_birth              0
education               0
marital_status          0
income                 24
kidhome                 0
teenhome                0
dt_customer             0
recency                 0
mntwines                0
mntfruits               0
mntmeatproducts         0
mntfishproducts         0
mntsweetproducts        0
mntgoldprods            0
numdealspurchases       0
numwebpurchases         0
numcatalogpurchases     0
numstorepurchases       0
numwebvisitsmonth       0
response                0
complain                0
dtype: int64
NANs
id                      0
year_birth              0
education               0
marital_status          0
income                 24
kidhome                 0
teenhome                0
dt_customer             0
recency                 0
mntwines                0
mntfruits               0
mntmeatproducts         0
mntfishproducts         0
mntsweetproducts        0
mntgoldprods            0
numdealspurcha

Обнаружено 24 пропуска по полю Income. 

In [41]:
print(f'Доля пропущенных данных в датасете: {data.query('income.isna()').shape[0] / data.shape[0] * 100:.2f}%')

Доля пропущенных данных в датасете: 1.07%


Заполнять пропуски не будем, удалим пропущенные значения (1% - допустимый уровень потерь при предобработке данных), чтобы не искажать данные, что может негативно повлиять на обучение моделей. Результат сохраним в новый датафрейм с очищенными данными.

In [42]:
# Создадим новый датафрейм в котором будут храниться очищенные данные для дальнейшей работы.

df = data.dropna()

### 2.3. Дубликаты данных

In [43]:
# Проверим количество явных дубликатов (полностью одинаковых строк)

df.duplicated().sum()

0

Явных дубликатов (полных совпадений) не обнаружено.

К неявным дубликатам могут относиться например варицации написания одной и той категории - лишние пробелы, ошибки в написании, разные регистры и т.д.

In [44]:
# Проверим наличие неявных дубликатов по столбцу education

df['education'].unique()

array(['Graduation', 'PhD', '2n Cycle', 'Master', 'Basic'], dtype=object)

В поле ``education`` ошибок нет.

Дадим пояснения по обозначениям уровней образования:
* Graduation - выпускник средней школы или колледжа / бакалавр
* PhD - доктор философии (высшая академическая степень)
* 2n Cycle - получение степени магистра (Болонская система)
* Master - магистр, степень следующая за обучение после бакалавриата
* Basic - базовое образование (начальная / средняя школа)

In [46]:
# Проверим наличие неявных дубликатов по столбцу education

df['marital_status'].unique()

array(['Divorced', 'Single', 'Married', 'Together', 'Widow', 'YOLO',
       'Alone', 'Absurd'], dtype=object)

В поле `marital_status` ошибок нет.

Дадим пояснения по обозначениям статусов семейного положения:
* Divorced - разведен
* Single - человек, который не состоит в браке и не имеет партнёра
* Married - женат / замужем
* Together - человек состоит в отношениях с другим человеком (без заключения брака)
* Widow - вдова
* YOLO - You Only Live Once - человек предпочитающий не связывать себя серьезными отношениями
* Alone - человек, который не находится в отношениях и предпочитает проводить время в одиночестве, это может быть связано с выбором или обстоятельствами. 
* Absurd - "все сложно"

В данной структуре статусов видим несколько дублирующих по смыслу понятий - так например статусы Divorced, Single, Widow, Alone означают одно и тоже состояние, но в результате разных обстоятельств. На следующем этапе предобработки создадим дополнительный признак, который объединит схожие по смыслу статусы.

In [54]:
# Также проверим наличие неявных дубликатов по связке полей year_birth, education, marital_status, income - основным полям 
# помимо id по которым можно идентифицировать похожих клиентов

df.duplicated(subset=['year_birth', 'education', 'marital_status', 'income']).sum()

201

Найден 201 дубликат, посмотрим на найденные совпадения.

In [56]:
dubls = df[df.duplicated(subset=['year_birth', 'education', 'marital_status', 'income'], keep=False)]
dubls.head(20)

,id,year_birth,education,marital_status,income,kidhome,teenhome,dt_customer,recency,mntwines,mntfruits,mntmeatproducts,mntfishproducts,mntsweetproducts,mntgoldprods,numdealspurchases,numwebpurchases,numcatalogpurchases,numstorepurchases,numwebvisitsmonth,response,complain
8,4047,1954,PhD,Married,65324.0,0,1,2014-11-01,0,384,0,102,21,32,5,3,6,2,9,4,0,0
9,9477,1954,PhD,Married,65324.0,0,1,2014-11-01,0,384,0,102,21,32,5,3,6,2,9,4,0,0
15,837,1977,Graduation,Married,54809.0,1,1,2013-11-09,0,63,6,57,13,13,22,4,2,1,5,4,0,0
16,10521,1977,Graduation,Married,54809.0,1,1,2013-11-09,0,63,6,57,13,13,22,4,2,1,5,4,1,0
26,5430,1956,Graduation,Together,54450.0,1,1,2012-09-14,0,454,0,171,8,19,32,12,9,2,8,8,0,0
27,8432,1956,Graduation,Together,54450.0,1,1,2012-09-14,0,454,0,171,8,19,32,12,9,2,8,8,0,0
68,1519,1972,PhD,Single,38578.0,1,1,2013-06-22,2,38,4,22,3,3,3,3,3,0,3,8,1,0
69,9080,1972,PhD,Single,38578.0,1,1,2013-06-22,2,38,4,22,3,3,3,3,3,0,3,8,0,0
80,9888,1969,Graduation,Together,68695.0,0,0,2014-06-25,3,458,81,356,106,50,40,1,4,4,7,2,0,0
81,4399,1969,Graduation,Together,68695.0,0,0,2014-06-25,3,458,81,356,106,50,40,1,4,4,7,2,0,0


По найденным данным действительно видно, что строки повторяются полностью, отличие только в id клиента. Это может говорить о технической ошибке при сборе данных или регистрации клиентов.

Удалим найденные дубликаты из очищенного датасета.

In [57]:
df = df.drop_duplicates(subset=['year_birth', 'education', 'marital_status', 'income'])

In [62]:
# Проверим изменение объема данных относительно первоначального датасета

print(f'Изменение объема данных: {(df.shape[0] / data.shape[0] * 100) - 100 :.2f}%')

Изменение объема данных: -10.04%


В результате выполненной предобработки, объем данных сократился на 10%, что является умеренным уровнем потерь.

### 2.4. Дообогащение данных

С учетом имеющихся данных дообогатим датасет следующими признаками:
* возраст клиента на момент регистрации в программе лояльности
* обобщенный признак наличия детей
* обобщающий семейный статус, убирающий повторяющиеся по смыслу категории
* общая сумма покупок по основным товарным категориям
* общее количество покупок по разным каналам
* средний чек покупки (общая сумма / общее кол-во покупок)

In [63]:
# Добавим поле с возрастом клиента на момент регистрации в программе лояльности

df['registration_age'] = df['dt_customer'].dt.year - df['year_birth']

In [68]:
# Добавим поле с обобщенным признаком наличия детей 1 - дети есть, 0 - детей нет

df['is_parent'] = df.apply(lambda x: 1 if (x['kidhome'] > 0 or x['teenhome'] > 0) else 0, axis=1)

In [73]:
# Добавим поле с обобщенным признаком семейного статуса

# Создадим список статусов соответсвующих по смыслу Single
single_list = ['Divorced', 'Single', 'Widow', 'Alone']

# Добавим столбец с признаком унифицирующим семейный статус
df['mstatus_unified'] = df.apply(lambda x: 'Single' if x['marital_status'] in single_list else x['marital_status'], axis=1)

In [76]:
# Добавим поле с общей суммой покупок в основных товарных категориях (без золотой полки, т.к. сумма может задублироваться)

df['mnt_total'] = (
    df['mntwines'] +
    df['mntfruits'] +
    df['mntmeatproducts'] + 
    df['mntfishproducts'] +
    df['mntsweetproducts']
)

In [79]:
# Добавим поле с общим количеством покупок (без покупок по скидке, чтобы не было задвоения по каналам)

df['total_purchases'] = (
    df['numwebpurchases'] +
    df['numcatalogpurchases'] +
    df['numstorepurchases']
)

In [81]:
# Добавим поле со средним чеком покупки

df['avg_check'] = df['mnt_total'] / df['total_purchases']

In [83]:
# Проверим полученный результат

df.head()

,id,year_birth,education,marital_status,income,kidhome,teenhome,dt_customer,recency,mntwines,mntfruits,mntmeatproducts,mntfishproducts,mntsweetproducts,mntgoldprods,numdealspurchases,numwebpurchases,numcatalogpurchases,numstorepurchases,numwebvisitsmonth,response,complain,registration_age,is_parent,mstatus_unified,mnt_total,total_purchases,avg_check
0,1826,1970,Graduation,Divorced,84835.0,0,0,2014-06-16,0,189,104,379,111,189,218,1,4,4,6,1,1,0,44,0,Single,972,14,69.428571
1,1,1961,Graduation,Single,57091.0,0,0,2014-06-15,0,464,5,64,7,0,37,1,7,3,7,5,1,0,53,0,Single,540,17,31.764706
2,10476,1958,Graduation,Married,67267.0,0,1,2014-05-13,0,134,11,59,15,2,30,1,3,2,5,2,0,0,56,1,Married,221,10,22.100000
3,1386,1967,Graduation,Together,32474.0,1,1,2014-11-05,0,10,0,1,0,0,0,1,1,0,2,7,0,0,47,1,Together,11,3,3.666667
4,5371,1989,Graduation,Single,21474.0,1,0,2014-08-04,0,6,16,24,11,0,34,2,3,1,2,7,1,0,25,1,Single,57,6,9.500000


Все 6 новых признаков добавлены.

Так как поле с идентификатором клиента не несет никакой смысловой нагрузки, мы его удалим из очищенного датасета.

In [85]:
# Удалим колонку с идентификаторами клиентов

df = df.drop(columns='id')

### 2.5. Выводы по предобработке данных

В результате выполненной предобработки:
* Поле `dt_customer` приведено к типу даты
* Удалено 24 записи с пропусками по полю income
* Удален 201 дубликат, обнаружена ошибка с задвоенными записями и разными идентификаторами пользователей, что могло быть связано с технической ошибкой при сборе данных или при регистрации клиентов.
* Объем данных сократился на 10% (умеренные потери)
* Добавлены шесть новых признаков:
    * `registration_age` - возраст клиента на момент регистрации в программе лояльности
    * `is_parent` - обобщенный признак наличия детей
    * `mstatus_unified` - обобщающий семейный статус, убирающий повторяющиеся по смыслу категории
    * `mnt_total` - общая сумма покупок по основным товарным категориям
    * `total_purchases` - общее количество покупок по разным каналам
    * `avg_check` - средний чек покупки (общая сумма / общее кол-во покупок)

## 3. Исследовательский анализ данных

## 4. Портрет покупателя

## 5. Кластеризация покупателей

## 6. Выбор и обучение модели для определения вероятности покупки товара

## 7. Общие выводы по результатам работы